# Amsterdam: SQL queries, times series, reviews, verifications and licenses 

Let's import useful packages:

In [7]:
import gzip
import json
import csv
import sqlite3
import pandas as pd
import numpy as np
from scipy import stats

Visualization tools:

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import folium

In [3]:
import sys
# !{sys.executable} -m pip install -U folium

In this last analysis, we create a database in Python using *sqlite3* importing a DataFrame using Pandas. The step 1 is create a database called BookingsDB.

In [8]:
conn = sqlite3.connect('BookingsDB.db') 

In [9]:
c = conn.cursor()

In [11]:
read_listing = pd.read_csv('../Data/raw/listings.csv.gz', 
                      compression='gzip',
                      error_bad_lines=False, 
                      low_memory=False)

In [12]:
read_listing.to_sql('LISTINGS', conn, if_exists='append', index = False) # Insert the values from the csv file into the table 'CLIENTS' 

In [19]:
c.execute('''
SELECT id FROM LISTINGS LIMIT 100
''')

In [20]:
print(c.fetchall())

[(2818,), (20168,), (25428,), (27886,), (28871,), (29051,), (31080,), (41125,), (42970,), (43109,), (43980,), (46386,), (47061,), (48076,), (49552,), (50515,), (50518,), (50523,), (50570,), (52490,), (53067,), (53671,), (53692,), (55256,), (55621,), (55703,), (55807,), (55868,), (56879,), (58211,), (62015,), (62801,), (63872,), (64736,), (64769,), (67841,), (69042,), (70598,), (73208,), (73917,), (75382,), (75444,), (76668,), (80635,), (82482,), (82748,), (83498,), (84057,), (91535,), (96048,), (97221,), (107195,), (108890,), (118097,), (118117,), (120872,), (130390,), (137026,), (137901,), (141708,), (148132,), (155548,), (162467,), (165833,), (168769,), (169356,), (170223,), (171054,), (171498,), (171631,), (174592,), (175989,), (179528,), (182839,), (188347,), (188483,), (189754,), (190585,), (190943,), (193038,), (198307,), (199844,), (200461,), (201322,), (203658,), (205759,), (212050,), (213721,), (214531,), (219276,), (221922,), (221943,), (222516,), (226121,), (230710,), (23618

A brief inspection of columns enable us to split them according to the following **different categories of information**:

1. **General information of booking**: id, name, summary, description, space, experiences_offered, neighborhood overview, notes, transit, access, interaction, house_rules. To access that, select columns from `id` to `house_rules`.

2. **Host information**: id, name, location, about, response time, response rate, acceptance rate, listings count, verifications. In this case, select columns from `host_id` to `host_identity_verified`.

3. All about the **neighborhood**: city, state, market, cleansed. Extract this information selecting columns from `street` to `is_location_exact`.

4. Details of **property and conditions** are between the columns `property_type` and `maximum_nights_avg_ntm`.

5. **Calendar updates and availability** the next 30, 60 and 90 days, for instances. You can find this information between the columns `calendar_updated` and `calendar_last_scraped`.

6. **Reviews**: first and last review, number of reviews, scores rating, scores per accuracy, cleanliness, checkin, communication, location and value. Columns from `number of reviews` to `review_scores_value`.

7. **Verifications and licenses**: require guest profile picture, phone verification, cancellation policy, instant bookeable. Columns from `requires_license` to `require_guest_phone_verification`.

8. **Host listings counts** and reviews per month. Select from `calculated_host_listings_count` to `reviews_per_month`.

## 5. Calendars